<a href="https://colab.research.google.com/github/gabrielvieiraf/Algodom/blob/master/AltaPerformance/Introdu%C3%A7%C3%A3o/Fundamentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook é dedicado para aqueles que visam desenvolver suas habilidades em Python, para quem deseja consultar alguns conceitos perdidos na memória e para programadores que vêm de outra linguegem, principalmente de mais baixo nível, e se deparam com a pergunta: **Existe performance no Python?**.

Este notebook será baseado no livro *High Performance Python, 2nd Edition* e tem, por sua vez, comentários e alterações feitas por mim, **Gabriel Vieira Flores**, afim de deixar o conteúdo com um pouco mais de informação e didático para os falantes da língua portuguesa.

> # Fundamentos do Python de Alta Performance

> ## Colocando em prática

Para entender os fundamentos de uma programação de alta performance, é fundamental entender os conceitos computacionais por trás do funcionamento da máquina ao qual o código foi escrito.

Nessa seção, iremos explorar alguns problemas para ilustrar como soluções ideais podem funcionar e como o Python se aproxima dessas soluções.


> ## Computação Ideal vs Máquina Virtual do Python

> ### Computação Ideal

Para melhor compreender os conceitos, utilizaremos um código simples que verifica se um número é primo

In [3]:
import math

def checa_primo(numero):
  sqrtNumero = math.sqrt(numero)
  for i in range(2, int(sqrtNumero) + 1):
    if (numero / i).is_integer():
      return False
  return True

print(f"checa_primo(10,000,000) = {checa_primo(10_000_000)}")
# checa_primo(10,000,000) = False

print(f"checa_primo(10,000,019) = {checa_primo(10_000_019)}")
# checa_primo(10,000,019) = True

checa_primo(10,000,000) = False
checa_primo(10,000,019) = True


Vamos analisar este código usando nosso modelo abstrato de computação e então
faça comparações com o que acontece quando o Python executa esse código.

Como com qualquer abstração, negligenciaremos muitas das sutilezas do computador idealizado e da maneira como o Python executa o código. No entanto, este é geralmente um bom exercício para realizar antes de resolver um problema: 

**Pense sobre os componentes gerais do algoritmo e qual seria a melhor
forma de os componentes de computação se unirem para encontrar uma solução.**


> #### Computação Idealizada

Quando o código começa, temos o valor do número armazenado na RAM. Para calcular ```sqrtNumero```, **precisamos enviar o valor de ```numero``` para a CPU.**

Idealmente, poderíamos enviar o valor uma vez; ficaria armazenado dentro do
Cache L1 / L2 da CPU, e a CPU faria os cálculos e, em seguida, enviaria
os valores para a RAM para serem armazenados. Este cenário é ideal porque nós minimizaram o número de leituras do valor de ```numero``` da RAM,
em vez disso, opta por leituras do cache L1 / L2, que são muito mais rápidas.

Além disso, minimizamos o número de transferências de dados por meio do
**bus frontal**, usando o cache L1 / L2 que está conectado diretamente ao
CPU.

**Obs:** Em PCs, o barramento frontal (Front Side Bus ou FSB em inglês) é o barramento de **transferência de dados que transporta informação entre a UCP e o northbridge da placa-mãe.**

Alguns computadores também possuem um barramento traseiro (ou *backside bus*) o qual conecta a UCP à memória cache interna. Este barramento e a memória cache associada a ela podem ser acessados muito mais rapidamente do que a RAM do sistema através do barramento frontal.

> #### Dica: 

Este tema de manter os dados onde são necessários e movê-los o menos possível
é muito importante quando se trata de otimização.

O conceito de *“heavy data”* refere-se ao tempo e esforço necessários para mover os dados, que é algo que gostaríamos de evitar.

> #### Conceito de vetorização

Para o loop no código, em vez de enviar um valor de ```i``` de cada vez para o
CPU, gostaríamos de enviar o ```numero``` e vários valores de ```i``` para o
CPU **para verificar ao mesmo tempo**.

Isso é possível porque a CPU vetoriza operações sem nenhum custo adicional de tempo, o que significa que pode fazer vários cálculos independentes ao mesmo tempo. Então, queremos enviar ```numero``` para o cache da CPU, além de tantos valores de ```i``` quanto o cache pode conter.

Para cada um dos pares de ```numero / i```, vamos dividi-los e verificar se o resultado é um número inteiro; então enviaremos um sinal de volta indicando se algum dos valores era de fato um número inteiro.

Nesse caso, a função termina. Se não, repetimos. Desta forma, precisamos comunicar de volta apenas um resultado para muitos valores de ```i```, em vez de depender do barramento lento para cada valor. Esta aproveita a capacidade da CPU de vetorizar um cálculo ou executar um instrução em vários dados em um ciclo de *clock*.

> #### Exemplo:

In [7]:
import math
def checa_primo(numero):
  sqrtNumero = math.sqrt(numero)
  numeros = range(2, int(sqrtNumero)+1)
  for i in range(0, len(numeros), 5):
    # A linha seguinte não é válida para um código em Python
    resultado = (numero / numeros[i:(i + 5)]).is_integer()
    if any(resultado):
      return False
  return True

Aqui, configuramos o processamento de forma que a divisão e a verificação de
inteiros são feitos em um conjunto de cinco valores de ```i``` por vez. Se vetorizado corretamente, a CPU pode fazer esta linha em uma etapa em vez de fazer uma
cálculo para cada ```i```.

> ### Máquina Virtual do Python

O interpretador Python trabalha muito para tentar abstrair os elementos de computação subjacentes que estão sendo usados. Em nenhum momento um programador precisa se preocupar em alocar memória para matrizes, como organizar isso na 
memória, ou em que sequência está sendo enviado para a CPU.

Isso é um benefício do Python, pois permite que você se concentre nos algoritmos que estão sendo implementados.**No entanto, isso tem um custo enorme de desempenho.**

É importante perceber que, em sua essência, o Python está realmente executando um conjunto de instruções muito otimizadas. O truque, no entanto, é fazer com que o Python os execute na sequência correta para obter um melhor desempenho.

Por exemplo, é muito fácil ver que, no exemplo a seguir, ```pesquisa_rapida```
será executado mais rápido do que ```pesquisa_lenta``` simplesmente porque ignora os cálculos desnecessários que resultam de não terminar o loop antes do tempo, embora ambas as soluções têm tempo de execução O (n).

No entanto, as coisas podem ficar complicadas ao lidar com tipos derivados, métodos Python especiais ou módulos terceiros.

Por exemplo, você pode dizer imediatamente qual função será
mais rápido: ```pesquisa_desconhecida1``` ou ```pesquisa_desconhecida2```?

In [2]:
def pesquisa_rapida(palheiro, agulha):
  for item in palheiro:
    if item == agulha:
      return True
  return False

def pesquisa_lenta(palheiro, agulha):
  valorRetorno = False
  for item in palheiro:
    if item == agulha:
      valorRetorno = True
  return valorRetorno

def pesquisa_desconhecida1(palheiro, agulha):
  return any((item == agulha for item in palheiro))

def pesquisa_desconhecida2(palheiro, agulha):
  return any([item == agulha for item in palheiro])

In [47]:
import time

palheiro = ['palha','palha','palha','palha','palha','palha',
            'palha','palha','palha','palha','palha','palha','agulha']
agulha = 'agulha'

def tempo_de_execucao(funcao):
  inicio = time.time()
  funcao
  fim = time.time()
  print(f"\nTempo de execução:\n{fim-inicio}")


tempo_de_execucao(pesquisa_rapida(palheiro, agulha))
tempo_de_execucao(pesquisa_lenta(palheiro, agulha))
tempo_de_execucao(pesquisa_desconhecida1(palheiro, agulha))
tempo_de_execucao(pesquisa_desconhecida2(palheiro,agulha)) 


Tempo de execução:
2.384185791015625e-07

Tempo de execução:
4.76837158203125e-07

Tempo de execução:
2.384185791015625e-07

Tempo de execução:
4.76837158203125e-07


> #### Recapitulando

Identificar regiões lentas de código por meio de criação de perfil e encontrar maneiras mais eficientes de fazer os mesmos cálculos é **semelhante a encontrar essas operações inúteis e removê-las**; o resultado final é o mesmo, mas o número de cálculos e transferências de dados é reduzido drasticamente.

Um dos impactos dessa camada de abstração é que a vetorização não é imediatamente alcançável. Nossa rotina de número primo inicial executará uma iteração do loop por valor de ```i``` em vez de combinar várias iterações.

No entanto, olhando para o exemplo de vetorização abstraída, vemos que não é um código Python válido, pois não podemos dividir um ```float``` por uma lista. Bibliotecas como o numpy ajudarão nesta situação, adicionando a capacidade de
fazer operações matemáticas vetorizadas.

Além disso, em nenhum momento há uma oportunidade de mudar o layout de uma estrutura de dados diretamente na memória, o que significa que uma transferência no *bus* pode não conter todas as informações relevantes
informações para um cálculo, mesmo que todas possam caber na largura do *bus*.

> #### Mais problemas no Mundo Real

Um outro problema vem dos tipos dinâmicos do Python
e a linguagem não está sendo compilada. Como muitos programadores C têm
aprendido ao longo dos anos, **o compilador geralmente é mais inteligente do que você.**

Ao compilar código estático, o compilador pode fazer muitos truques para
mude a forma como as coisas são organizadas e como a CPU executará certas instruções para otimizá-las. Python, no entanto, **não é compilado**: para
piorar as coisas, ele tem tipos dinâmicos, o que significa que inferir qualquer
as oportunidades possíveis para otimizações algorítmicas são drasticamente
mais difícil, pois a funcionalidade do código pode ser alterada durante o tempo de execução. 

> #### Ok, mas Python é Compilado ou Interpretado ?!!!

A implementação mais comum do Python é o CPython, escrita em C. CPython é uma linguagem interpretada com certo nível de compilação. CPython compila** código fonte Python para bytecode, e depois *interpreta este bytecode, executando conforme progride.

Bytecode vs. Código de Máquina
É muito importante entender a diferença entre bytecode e código de máquina (ou nativo), talvez melhor ilustrado pelo exemplo:

C compila para código de máquina, que é depois executado diretamente no seu processador. Cada instrução instrui sua CPU a mover coisas por toda parte.
Java compila para bytecode, que depois é executado na Java Virtual Machine (JVM), uma abstração de um computador que executa programas. Cada instrução é então tratada pela JVM, que interage com seu computador.

Em termos breves: código de máquina é muito mais rápido, mas bytecode é mais portável e seguro.

Código de máquina parece diferente dependendo de sua máquina, mas bytecode se parece igual em todas as máquinas. Alguém pode dizer que código de máquina é otimizado para sua configuração.

Retornando ao CPython, o processo de toolchain é como segue:

CPython compila seu código fonte Python para bytecode.
Este bytecode é então executado na Máquina Virtual CPython.
Iniciantes assumem que Python é compilado por causa dos arquivos .pyc. Há alguma verdade nisto: o arquivo .pyc é bytecode compilado, que é depois interpretado. Então se você rodou seu código Python antes e tem o arquivo .pyc disponível, ele vai rodar mais rápido na segunda vez, já que não precisará recompilar o bytecode.

**trecho retirado do artigo : https://www.toptal.com/python/por-que-h-tantos-pythons**

> #### Continuando


Existem muitas maneiras de mitigar este problema, principalmente sendo o uso de Cython, que permite que o código Python seja compilado e permite ao usuário criar “Dicas” para o compilador sobre o quão dinâmico o código realmente é.

Finalmente, o GIL mencionado anteriormente pode prejudicar o desempenho se tentar paralelizar este código. Por exemplo, vamos supor que mudamos o código para usar múltiplos núcleos de CPU, de modo que cada núcleo obtém um pedaço dos números de 2 a sqrtN. Cada núcleo pode fazer seu cálculo para seu bloco de números, e então, quando todos os cálculos estiverem feitos, os núcleos podem comparar seus cálculos.

Embora percamos o término antecipado do loop, uma vez que cada núcleo não sabe se uma solução foi encontrada, podemos reduzir o número de verifica que cada núcleo tem que fazer (se tivéssemos M núcleos, cada núcleo teria que fazer
4 verificações sqrtN / M). No entanto, por causa do GIL, **apenas um núcleo pode ser usado por vez.** Isso significa que estaríamos efetivamente executando o mesmo código como a versão incomparável, mas não temos mais o encerramento antecipado.

Podemos evitar esse problema usando vários processos (com o módulo de multiprocessamento) em vez de vários threads, ou usando Cython ou funções estrangeiras. 

> #### CPython vs Cython

Os módulos Cython têm uma extensão de arquivo .pyx em vez de .py. O processo de construção do Cython os traduz em arquivos de origem C intermediários e os compila usando o compilador C do sistema.

O código Cython se parece com o código Python com declarações de tipo adicionadas. As declarações de tipo para variáveis, argumentos de função e valores de retorno são opcionais, mas adicioná-los fornece a Cython informações suficientes para traduzir em um código C mais otimizado.

Cython permite fácil vinculação a bibliotecas C e permite funções C nativas, que têm menos sobrecarga do que as funções Python quando são chamadas e, portanto, são executadas mais rapidamente. 

Comparando implementação, tempo de execução e velocidade, a implementação CPython + Cython é a mais rápida; é 44 vezes mais rápido do que a implementação do CPython. Esta é uma melhoria de velocidade impressionante, especialmente considerando que o código Cython está muito próximo do código Python original em seu design.

**trecho retirado do artigo: https://www.cardinalpeak.com/blog/faster-python-with-cython-and-pypy-part-2**

> ## Ok, entendi! Mas por que devo programar em Python?

Python é altamente expressivo e fácil de aprender. Novos programadores rapidamente podem fazer muito em um curto espaço de tempo. Muitas bibliotecas agrupam ferramentas escritas em outras linguagens para tornar mais fácil chamar outros sistemas; por exemplo, o sistema de aprendizado de máquina ```scikit-learn``` envolve LIBLINEAR e LIBSVM (ambos escritos em C), e a biblioteca numpy inclui BLAS e outras bibliotecas C e Fortran. Como resultado, o código Python que utiliza adequadamente esses módulos pode ser tão rápido como código C comparável.

**No Python,  muitas ferramentas importantes e
Bibliotecas estáveis ​​são integradas.**

* **array:**
  Matrizes com eficiência de memória para tipos primitivos;

* **math:**
Operações matemáticas básicas, incluindo estatísticas simples;

* **sqlite3:**
Um wrapper em torno do mecanismo de armazenamento baseado em arquivo SQL;

* **collections:**
Uma grande variedade de objetos, incluindo um deque, contador e dicionário;

* **asyncio:**
Suporte simultâneo para tarefas vinculadas a E / S usando sintaxe assíncrona e de espera.

E muitas outras!

A distribuição padrão, disponível em http://python.org pipenv, pyenv e virtualenv para ambientes Python simples, leves e portáteis. 

**Anaconda da Anaconda Inc.**, um ambiente com foco científico, um ambiente semelhante ao Matlab que inclui um ambiente de desenvolvimento integrado (IDE) IPython, um shell Python interativo amplamente usado por cientistas e desenvolvedores **Jupyter Notebook**, uma extensão baseada em navegador para IPython, pesadamente usado para ensino e demonstrações **como essa aqui :)**

Um dos principais pontos fortes do Python é que ele permite a **prototipagem rápida de uma ideia**. Devido à **grande variedade de bibliotecas de suporte**, é fácil testar se uma ideia é viável, mesmo que a primeira implementação possa ser bastante escamada.

Em geral, é sensato levantar a questão: **“Se nosso sistema funcionar mais rápido,
Será que vamos, como equipe, correr mais devagar no longo prazo? ”** Sempre é possível extrair mais desempenho de um sistema se houver horas de trabalho suficientes dentro adquiridos, mas isso pode levar a otimizações frágeis e mal compreendidas que no final das contas tropeçou na equipe.

Um exemplo pode ser a introdução do Cython, uma abordagem baseada em compilador para anotar o código Python com tipos C-like para que o código transformado possa ser compilado usando um compilador C. Enquanto na velocidade, os ganhos podem ser impressionantes (muitas vezes atingindo velocidades semelhantes às de C com relativo esforço), o custo de suporte a esse código aumentará.

Em particular, pode ser mais difícil oferecer suporte a este novo módulo, pois os membros da equipe precisarão de uma certa maturidade em sua capacidade de programação para compreender alguns dos compensações que ocorreram ao deixar a máquina virtual Python queintroduziu o aumento de desempenho.